# Create Document Knowledgebase with Azure OpenAI and Cognitive Search
This code demonstrates how to use Azure Cognitive Search with OpenAI and Azure Python SDK to extract information from text content in json file, add vectors and index in Azure Cog Search.

Input data is in .\data folder

## Prerequisites
To run the code, install the following packages. Please use the latest pre-release version `pip install azure-search-documents==11.4.0b6`.

- > ! pip install azure-search-documents==11.4.0b6
- > ! pip install openai

## Load all the AOAI API keys and model parameters

In [1]:
import os
import aoai

MY_AOAI_ENDPOINT = os.environ['OPENAI_API_ENDPOINT']
MY_AOAI_KEY = os.environ['OPENAI_API_KEY']
MY_AOAI_VERSION = os.environ['OPENAI_API_VERSION']
MY_GPT_ENGINE = os.environ['OPENAI_API_ENGINE']
MY_AOAI_EMBEDDING_ENGINE = 'text-embedding-3-small'

status, client = aoai.setupOpenai(
                        aoai_endpoint=MY_AOAI_ENDPOINT,
                        aoai_api_key=MY_AOAI_KEY,
                        aoai_version=MY_AOAI_VERSION
                 )
if status == True:
    print("AOAI setup succeeded")
else:
    print("AOAI setup failed")


Got OPENAI API Key from environment variable
AOAI setup succeeded


## Create embeddings
Read your data, generate OpenAI embeddings and export to a format to insert your Azure Cognitive Search index:

In [9]:
import json
from tenacity import retry, wait_random_exponential, stop_after_attempt  

# Generate Document Embeddings using OpenAI Ada 002

# TODO: Read from Blob Store
# Assuming you are running notebook from the notebook folder
MY_INPUT_DATA_FILE = r'..\..\..\data\sample-azure-service-docs\text-sample.json'
MY_OUTPUT_DATA_FILE = r'..\..\..\data\sample-azure-service-docs\text-sample-with-vectors.json'

In [ ]:
# Read the text-sample.json
with open(MY_INPUT_DATA_FILE, 'r', encoding='utf-8') as file:
    input_data = json.load(file)

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Generate embeddings for title and content fields
def add_embedding(text_data):
    return aoai.generate_embedding(the_engine = MY_AOAI_EMBEDDING_ENGINE,
                              the_text = text_data)
for item in input_data:
    title = item['title']
    content = item['content']
    title_embeddings = add_embedding(title)
    content_embeddings = add_embedding(content)
    item['titleVector'] = title_embeddings
    item['contentVector'] = content_embeddings

# Output embeddings to docVectors.json file
with open(MY_OUTPUT_DATA_FILE, "w") as f:
    json.dump(input_data, f)

## Authenticate to Azure Cognitive Search and connect

In [3]:
import cog_search

cogSearchCredential = cog_search.getCogSearchCredential()


Got Azure Cognitive Search ADMIN API Key from environment variable


In [5]:
from azure.search.documents.indexes import SearchIndexClient

# Create a search index
MY_COG_SEARCH_ENDPOINT = 'https://tr-docai-cog-search.search.windows.net'
MY_COG_SEEARCH_INDEX_NAME = 'sample-azure-service-docs-index'

## Import all cognitive search packages

In [6]:
from azure.search.documents import SearchClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    VectorSearchAlgorithmConfiguration,  
)  

## Create your search index
Create your search index schema and vector search configuration:

In [7]:
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchableField(name="category", type=SearchFieldDataType.String,
                    filterable=True),
    SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
]

vector_search = VectorSearch(
    algorithm_configurations=[
        VectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            kind="hnsw",
            hnsw_parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 500,
                "metric": "cosine"
            }
        )
    ]
)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        title_field=SemanticField(field_name="title"),
        prioritized_keywords_fields=[SemanticField(field_name="category")],
        prioritized_content_fields=[SemanticField(field_name="content")]
    )
)

# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])


# create an index client connection
index_client = SearchIndexClient(endpoint=MY_COG_SEARCH_ENDPOINT, 
                                 credential=cogSearchCredential)
# delete any existing index first to have a clean slate
index_client.delete_index(MY_COG_SEEARCH_INDEX_NAME)
# Create the search index with the semantic settings
index = SearchIndex(name=MY_COG_SEEARCH_INDEX_NAME, fields=fields,
                    vector_search=vector_search, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')


 sample-azure-service-docs-index created


## Insert text and embeddings into Azure Cognitive Search Index
Add texts and metadata from the JSON data to the vector store:

In [10]:
# Upload some documents to the index
with open(MY_OUTPUT_DATA_FILE, 'r') as file:  
    documents = json.load(file)  
search_client = SearchClient(
                    endpoint=MY_COG_SEARCH_ENDPOINT, 
                    index_name=MY_COG_SEEARCH_INDEX_NAME, 
                    credential=cogSearchCredential)
result = search_client.upload_documents(documents)  
print(f"Uploaded {len(documents)} documents") 

Uploaded 108 documents


# Test different query types (full text, vector, semantic, cross-field, hybrid) to check if you can query Cog Search for content

## Perform a Vector Similarity Search

In [12]:
# Pure Vector Search multi-lingual (e.g 'tools for software development' in Dutch)  
query = "tools voor softwareontwikkeling"  
  
search_client = SearchClient(
                    endpoint=MY_COG_SEARCH_ENDPOINT, 
                    index_name=MY_COG_SEEARCH_INDEX_NAME, 
                    credential=cogSearchCredential)

results = search_client.search(  
    search_text=None,  
    vector=aoai.generate_embedding(
                    the_engine=MY_AOAI_EMBEDDING_ENGINE,
                    the_text=query), 
    top_k=1,  
    vector_fields="contentVector",
    select=["title", "content", "category"],
)  
  
for result in results:  
    print(f"Title: {result['title']}\n")  
    print(f"Score: {result['@search.score']}\n")  
    print(f"Content: {result['content']}\n")  
    print(f"Category: {result['category']}\n")  

Title: Azure DevOps

Score: 0.80402017

Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.

Category: Developer Tools



## Perform a Semantic Hybrid Search

<font color=red>Make sure Semantic Search is enabled for this Azure Cognitive Search instance</font>

In [10]:
# Semantic Hybrid Search
# note the incorrect spelling here - 'sarch'
query = "what is azure sarch?"

search_client = SearchClient(
                    endpoint=MY_COG_SEARCH_ENDPOINT, 
                    index_name=MY_COG_SEEARCH_INDEX_NAME, 
                    credential=cogSearchCredential)

results = search_client.search(
    search_text=query,
    vector=aoai.generate_embedding(
                    the_engine=MY_AOAI_EMBEDDING_ENGINE,
                    the_text=query), 
    top_k=1,  
    vector_fields="contentVector",
    select=["title", "content", "category"],
    query_type="semantic", 
    query_language="en-us", 
    semantic_configuration_name='my-semantic-config', 
    query_caption="extractive", 
    query_answer="extractive",
    top=1
)

#TODO Get the Category name instead of the key
semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer highlights: {answer.highlights}\n")
    else:
        print(f"Semantic Answer text: {answer.text}\n")
    print(f"Semantic Answer Score: {answer.score}\n")
    print(f"Category: {answer.key}\n")

print("---NOW SEE THE REGULAR SEARCH RESULT BELOW AND NOTICE IT IS NOT AS GOOD AS ABOVE---\n")
for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}\n")  
    print(f"Content: {result['content']}\n")
    print(f"Category: {result['category']}\n")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

Semantic Answer highlights: Azure Cognitive Search is<em> a fully managed search-as-a-service that enables you to build rich search experiences for your applications.</em> It provides features like full-text search, faceted navigation, and filters. Azure Cognitive Search supports various data sources, such as Azure SQL Database, Azure Blob Storage, and Azure Cosmos DB.

Semantic Answer Score: 0.9462890625

Category: 40

---NOW SEE THE REGULAR SEARCH RESULT BELOW AND NOTICE IT IS NOT AS GOOD AS ABOVE---

Title: Azure Stack Edge
Score: 0.009999999776482582

Content: Azure Stack Edge is a managed, edge computing appliance that enables you to run Azure services and AI workloads on-premises or at the edge. It provides features like hardware-accelerated machine learning, local caching, and integration with Azure IoT Hub. Azure Stack Edge supports various Azure services, such as Azure Functions, Azure Machine Learning, and Azure Kubernetes Service. You can use Azure Stack Edge to build edge c

## Perform a Cross-Field Vector Search

In [11]:
# Cross-Field Vector Search
query = "tools for software development"  
  
search_client = SearchClient(
                    endpoint=MY_COG_SEARCH_ENDPOINT, 
                    index_name=MY_COG_SEEARCH_INDEX_NAME, 
                    credential=cogSearchCredential)
  
results = search_client.search(  
    search_text=None,  
    vector=aoai.generate_embedding(
                    the_engine=MY_AOAI_EMBEDDING_ENGINE,
                    the_text=query), 
    top_k=1,  
    vector_fields="titleVector, contentVector",
    select=["title", "content", "category"],
)  
  
for result in results:  
    print(f"Title: {result['title']}\n")  
    print(f"Score: {result['@search.score']}\n")  
    print(f"Content: {result['content']}\n")  
    print(f"Category: {result['category']}\n")  


Title: Azure DevOps

Score: 0.03333333507180214

Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.

Category: Developer Tools



## Perform a Pure Vector Search with a filter

In [12]:
# Pure Vector Search with Filter
query = "tools for software development"  
  
search_client = SearchClient(
                    endpoint=MY_COG_SEARCH_ENDPOINT, 
                    index_name=MY_COG_SEEARCH_INDEX_NAME, 
                    credential=cogSearchCredential)
  

results = search_client.search(  
    search_text=None,  
    vector=aoai.generate_embedding(
                    the_engine=MY_AOAI_EMBEDDING_ENGINE,
                    the_text=query), 
    top_k=1,  
    vector_fields="contentVector",
    filter="category eq 'Developer Tools'",
    select=["title", "content", "category"]
)  
  
for result in results:  
    print(f"Title: {result['title']}\n")  
    print(f"Score: {result['@search.score']}\n")  
    print(f"Content: {result['content']}\n")  
    print(f"Category: {result['category']}\n")  


Title: Azure DevOps

Score: 0.82971567

Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.

Category: Developer Tools



## Perform a Hybrid Search

In [13]:
# Hybrid Search
query = "scalable storage solution"  
  
search_client = SearchClient(
                    endpoint=MY_COG_SEARCH_ENDPOINT, 
                    index_name=MY_COG_SEEARCH_INDEX_NAME, 
                    credential=cogSearchCredential)
  
results = search_client.search(  
    search_text=query,  
    vector=aoai.generate_embedding(
                    the_engine=MY_AOAI_EMBEDDING_ENGINE,
                    the_text=query), 
    top_k=1,  
    vector_fields="contentVector",
    select=["title", "content", "category"],
    top=1
)  
  
for result in results:  
    print(f"Title: {result['title']}\n")  
    print(f"Score: {result['@search.score']}\n")  
    print(f"Content: {result['content']}\n")  
    print(f"Category: {result['category']}\n")  


Title: Azure Blob Storage

Score: 0.03205128386616707

Content: Azure Blob Storage is a scalable, durable, and high-performance object storage service for unstructured data. It provides features like data redundancy, geo-replication, and fine-grained access control. Blob Storage supports various data types, such as images, documents, and videos. You can use Blob Storage to store and manage your data, build data lakes, and develop big data analytics solutions. It also integrates with other Azure services, such as Azure CDN, Azure Functions, and Azure Machine Learning.

Category: Storage

